In [1]:
#This is SQL analysis of Superstore Sales dataset

In [2]:
import sqlite3
import pandas as pd

sales_data = pd.read_csv('../data/Superstore_sales_sample.csv', encoding = 'cp1252')

In [7]:
#changing column names to standard format in lowercase, underscores only
sales_data.columns = (
    sales_data.columns
    .str.lower()
    .str.replace(' ', '_' )
    .str.replace('-', '_')
)

In [8]:
#convert order and ship dates to datetime
sales_data['order_date'] = pd.to_datetime(sales_data['order_date'])
sales_data['ship_date'] = pd.to_datetime(sales_data['ship_date'])
sales_data[['order_date','ship_date']].dtypes

order_date    datetime64[ns]
ship_date     datetime64[ns]
dtype: object

In [10]:
#profit margin 
sales_data['profit_margin'] = sales_data['profit']/sales_data['sales']
#delivery days in numeric
sales_data['delivery_days'] = ( sales_data['ship_date'] - sales_data['order_date'] ).dt.days

In [36]:
#sorting delivery days as Fast, Medium or Slow
sales_data['delivery_speed'] = 'Unknown'

sales_data.loc[sales_data['delivery_days'] <= 1, 'delivery_speed'] = 'Fast'
sales_data.loc[
    (sales_data['delivery_days'] > 1) & (sales_data['delivery_days'] <= 3),
    'delivery_speed'
] = 'Medium'
sales_data.loc[sales_data['delivery_days'] >= 4, 'delivery_speed'] = 'Slow'


In [15]:
#connect to sqlite database
conn = sqlite3.connect('../sql/superstore.db')

In [17]:
#write dataframe to sql
sales_data.to_sql('sales', conn, if_exists='replace', index = False) #if table named sales already exist, then replace it. 
#And do not export pandas index as column


9994

In [20]:
pd.read_sql_query( 'SELECT COUNT (*) FROM sales' , conn )

,COUNT (*)
0,9994


In [33]:
#Which regions generate the most sales and profit?
query = """
   SELECT region, SUM(sales) AS total_sales, SUM(profit) AS total_profit
   FROM sales 
   GROUP BY region
   ORDER BY total_sales DESC
"""
pd.read_sql_query(query, conn)


,region,total_sales,total_profit
0,West,725457.8245,108418.4489
1,East,678781.2400,91522.7800
2,Central,501239.8908,39706.3625
3,South,391721.9050,46749.4303


In [34]:
sales_data.head(3)

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country,city,...,category,sub_category,product_name,sales,quantity,discount,profit,profit_margin,delivery_days,delivery_speed
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.0,41.9136,0.16,3,Medium
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.94,3,0.0,219.5820,0.30,3,Medium
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.62,2,0.0,6.8714,0.47,4,Slow


In [35]:
#Which product categories are most profitable relative to their sales?
query = """
   SELECT category, SUM(sales) AS total_sales, SUM(profit) AS total_profit,  SUM(profit)/SUM(sales) as weighted_profit_margin
   FROM sales 
   GROUP BY category
   ORDER BY weighted_profit_margin DESC
"""
pd.read_sql_query(query, conn)

,category,total_sales,total_profit,weighted_profit_margin
0,Technology,836154.0330,145454.9481,0.173957
1,Office Supplies,719047.0320,122490.8008,0.170352
2,Furniture,741999.7953,18451.2728,0.024867


In [37]:
#Do Fast / Medium / Slow deliveries differ in profitability?
query = """
   SELECT delivery_speed, SUM(sales) AS total_sales, SUM(profit) AS total_profit,  SUM(profit)/SUM(sales) as weighted_profit_margin
   FROM sales 
   GROUP BY delivery_speed
   ORDER BY weighted_profit_margin DESC
"""
pd.read_sql_query(query, conn)

,delivery_speed,total_sales,total_profit,weighted_profit_margin
0,Medium,5.731254e+05,79994.0330,0.139575
1,Slow,1.531192e+06,183475.7933,0.119825
2,Fast,1.928830e+05,22927.1954,0.118866


In [38]:
#connecting Power BI to SQLite
#export final analysis table 
df = pd.read_sql_query("SELECT * FROM sales", conn)
df.to_csv('../data/superstore_final_df.csv', index = False)

In [39]:
sales_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   row_id          9994 non-null   int64         
 1   order_id        9994 non-null   object        
 2   order_date      9994 non-null   datetime64[ns]
 3   ship_date       9994 non-null   datetime64[ns]
 4   ship_mode       9994 non-null   object        
 5   customer_id     9994 non-null   object        
 6   customer_name   9994 non-null   object        
 7   segment         9994 non-null   object        
 8   country         9994 non-null   object        
 9   city            9994 non-null   object        
 10  state           9994 non-null   object        
 11  postal_code     9994 non-null   int64         
 12  region          9994 non-null   object        
 13  product_id      9994 non-null   object        
 14  category        9994 non-null   object        
 15  sub_